In [78]:
import pandas as pd 
import numpy as np 

from scipy import sparse
import torch
from torch import nn
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans



In [79]:
X = np.loadtxt('Data/dblp.txt')
y = np.loadtxt('Data/dblp_label.txt', dtype = int)

In [80]:
X.shape

(4057, 334)

In [81]:
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [82]:
print(y)

[1 3 0 ... 3 3 2]


In [83]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [84]:
n = X.shape[0]

In [85]:
n_input = 50

pca = PCA(n_components=n_input)

X_pca = pca.fit_transform(X)

X = torch.Tensor(X_pca)

In [86]:
edges = np.loadtxt('Data/dblp_graph.txt', dtype=np.int32)

edges

array([[   1, 1833],
       [   2,   97],
       [   2, 1561],
       ...,
       [4055, 1286],
       [4055, 2202],
       [4055, 3007]], dtype=int32)

In [87]:

adj = sparse.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(n, n), dtype=np.float32)

adj = adj + sparse.eye(adj.shape[0])

degrees = np.array(adj.sum(axis=1)).flatten()

D_inv_sqrt = sparse.diags(np.power(degrees, -0.5))

adj = D_inv_sqrt.dot(adj).dot(D_inv_sqrt)


In [88]:

def to_torch_sparse_tensor(sparse_mx):

    sparse_mx = sparse_mx.tocoo().astype(np.float32)

    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))

    values = torch.tensor(sparse_mx.data, dtype=torch.float32)

    shape = torch.Size(sparse_mx.shape)

    return torch.sparse.FloatTensor(indices, values, shape)


In [89]:
adj = to_torch_sparse_tensor(adj)


In [90]:
# Load available Device 

device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mps_device = torch.device('mps')

In [91]:
# device = mps_device

device = torch.device('cpu')

In [92]:
adj

tensor(indices=tensor([[   0,    1,    1,  ..., 4055, 4055, 4056],
                       [   0,    1, 1833,  ..., 3007, 4055, 4056]]),
       values=tensor([1.0000, 0.5000, 0.4082,  ..., 0.2582, 0.2000, 1.0000]),
       size=(4057, 4057), nnz=11113, layout=torch.sparse_coo)

In [93]:
adj  = adj.to(device)
X = X.to(device)

label = y

In [94]:
class AE_Encoder(nn.Module):

    def __init__(self, in_channels, out_channels, hidden1_dim, hidden2_dim, hidden3_dim):
        super(AE_Encoder, self).__init__()

        self.enc1 = nn.Linear(in_channels, hidden1_dim)
        self.enc2 = nn.Linear(hidden1_dim, hidden2_dim)
        self.enc3 = nn.Linear(hidden2_dim, hidden3_dim)
        self.z_layer = nn.Linear(hidden3_dim, out_channels)
        self.act_fn = nn.LeakyReLU(0.2, inplace=True)
    
    def forward(self, x):
        z = self.act_fn(self.enc1(x))
        z = self.act_fn(self.enc2(z))
        z = self.act_fn(self.enc3(z))

        z = self.z_layer(z)
        
        return z
    

In [95]:

class AE_Decoder(nn.Module):

    def __init__(self, in_channels, out_channels, hidden1_dim, hidden2_dim, hidden3_dim):
        super(AE_Decoder, self).__init__()

        self.dec1 = nn.Linear(in_channels, hidden1_dim)
        self.dec2 = nn.Linear(hidden1_dim, hidden2_dim)
        self.dec3 = nn.Linear(hidden2_dim, hidden3_dim)
        self.xhat_layer = nn.Linear(hidden3_dim, out_channels)
        self.act_fn = nn.LeakyReLU(0.2, inplace=True)
    
    def forward(self, z):
        xhat = self.act_fn(self.dec1(z))
        xhat = self.act_fn(self.dec2(xhat))
        xhat = self.act_fn(self.dec3(xhat))

        xhat = self.xhat_layer(xhat)
        
        return xhat
    

In [96]:
class AutoEncoder(nn.Module):
    
    def __init__(self, n_input, n_z, n_ae_enc1, n_ae_enc2, n_ae_enc3, n_ae_dec1, n_ae_dec2, n_ae_dec3):
        super(AutoEncoder, self).__init__()

        self.encoder = AE_Encoder(n_input, n_z, n_ae_enc1, n_ae_enc2, n_ae_enc3)
        self.decoder = AE_Decoder(n_z, n_input, n_ae_dec1, n_ae_dec2, n_ae_dec3)

    def forward(self, x):
        z = self.encoder(x)
        xhat = self.decoder(z)

        return xhat, z

In [97]:
import torch.nn.functional as F

class GNNLayer(nn.Module):

    def __init__(self, in_channels, out_channels, active = False):
        super(GNNLayer, self).__init__()
        self.active = active
        self.fc = nn.Linear(in_channels, out_channels, )
        self.act_fn = nn.Tanh()

    def forward(self, x, adj):

        if self.active:
            support = self.act_fn(self.fc(x))
        else:
            support = self.fc(x)

        return torch.spmm(adj, support)

In [98]:
class IGAE_Encoder(nn.Module):

    def __init__(self, in_channels, out_channels, hidden1_dim, hidden2_dim):
        super(IGAE_Encoder, self).__init__()

        self.enc1 = GNNLayer(in_channels, hidden1_dim, active=True)
        self.enc2 = GNNLayer(hidden1_dim, hidden2_dim, active=True)
        self.enc3 = GNNLayer(hidden2_dim, out_channels, active=False)

        self.sigmoid_fn = nn.Sigmoid()

    def forward(self, x, adj):
        z = self.enc1(x, adj)
        z = self.enc2(z, adj)
        z = self.enc3(z, adj)

        adj_hat = self.sigmoid_fn(torch.mm(z, z.t()))

        return z, adj_hat


In [99]:
class IGAE_Decoder(nn.Module):

    def __init__(self, in_channels, out_channels, hidden1_dim, hidden2_dim):
        super(IGAE_Decoder, self).__init__()

        self.dec1 = GNNLayer(in_channels, hidden1_dim, active = True)
        self.dec2 = GNNLayer(hidden1_dim, hidden2_dim, active = True)
        self.dec3 = GNNLayer(hidden2_dim, out_channels, active=True)

        self.sigmoid_fn = nn.Sigmoid()

    def forward(self, z, adj):
        xhat = self.dec1(z, adj)
        xhat = self.dec2(xhat, adj)
        xhat = self.dec3(xhat, adj)

        adj_hat = self.sigmoid_fn(torch.mm(xhat, xhat.t()))

        return xhat, adj_hat



In [100]:
class IGAE(nn.Module):

    def __init__(self, n_input, n_z, n_igae_enc1, n_igae_enc2, n_igae_dec1, n_igae_dec2):
        super(IGAE, self).__init__()

        self.encoder = IGAE_Encoder(n_input, n_z, n_igae_enc1, n_igae_enc2)
        self.decoder = IGAE_Decoder(n_z, n_input, n_igae_dec1, n_igae_dec2)

    def forward(self, x, adj):

        z, adj_enc = self.encoder(x, adj)
        xhat, adj_dec = self.decoder(z, adj)

        adj_hat = adj_enc + adj_dec

        return z, xhat, adj_hat

In [101]:
class DFCN(nn.Module):
    
    def __init__(self, n_node, n_input, n_z, n_ae_enc1, n_ae_enc2, n_ae_enc3,n_ae_dec1, n_ae_dec2, n_ae_dec3, n_igae_enc1, 
                 n_igae_enc2, n_igae_dec1, n_igae_dec2, n_clusters, v = 1.0, device = None):
        
        super(DFCN, self).__init__()
        self.autoencoder = AutoEncoder(n_input, n_z, n_ae_enc1, n_ae_enc2, n_ae_enc3, n_ae_dec1, n_ae_dec2, n_ae_dec3)
        self.gae = IGAE(n_input, n_z, n_igae_enc1, n_igae_enc2,n_igae_dec1, n_igae_dec2)

        self.alpha = nn.Parameter(nn.init.constant_(torch.zeros(n_node, n_z), 0.5), requires_grad=True).to(device)

        self.cluster_layer = nn.Parameter(torch.Tensor(n_clusters, n_z), requires_grad = True)
        nn.init.xavier_normal_(self.cluster_layer.data)

        self.v = v
        self.beta = nn.Parameter(torch.zeros(1), requires_grad=True)

    def forward(self, x, adj):

        z_ae = self.autoencoder.encoder(x)
        z_igae, adj_igae = self.gae.encoder(x, adj)
        z_i = self.alpha * z_ae + (1 - self.alpha) * z_igae
        z_l = torch.spmm(adj, z_i)
        s = F.softmax(torch.mm(z_l, z_l.t()), dim = 1)
        z_g = torch.mm(s, z_l)
        z_tilde = self.beta * z_g + z_l

        xhat_ae = self.autoencoder.decoder(z_ae)
        xhat_igae, adj_igae_dec = self.gae.decoder(z_igae, adj)

        adj_hat = adj_igae + adj_igae_dec

        q = 1.0 / (1.0 + torch.sum(torch.pow((z_tilde).unsqueeze(1) - self.cluster_layer, 2), 2) / self.v)
        q = q.pow((self.v + 1.0) / 2.0)
        q = (q.t() / torch.sum(q, 1)).t()

        q_ae = 1.0 / (1.0 + torch.sum(torch.pow(z_ae.unsqueeze(1) - self.cluster_layer, 2), 2) / self.v)
        q_ae = q_ae.pow((self.v + 1.0) / 2.0)
        q_ae = (q_ae.t() / torch.sum(q_ae, 1)).t()

        q_igae = 1.0 / (1.0 + torch.sum(torch.pow(z_igae.unsqueeze(1) - self.cluster_layer, 2), 2) / self.v)
        q_igae = q_igae.pow((self.v + 1.0) / 2.0)
        q_igae = (q_igae.t() / torch.sum(q_igae, 1)).t()

        return xhat_ae, xhat_igae, adj_hat, z_ae, z_igae, q, q_ae, q_igae, z_tilde


In [102]:
n_z = 20
n_clusters = 4

n_ae_enc1 = 128
n_ae_enc2 = 256
n_ae_enc3 = 120

n_ae_dec1 = 120
n_ae_dec2 = 256
n_ae_dec3 = 128

n_igae_enc1 = 128
n_igae_enc2 = 256
n_igae_dec1 = 256
n_igae_dec2 = 128


model = DFCN(X.size()[0], n_input, n_z, n_ae_enc1, n_ae_enc2, n_ae_enc3, n_ae_dec1, n_ae_dec2, n_ae_dec3, 
             n_igae_enc1, n_igae_enc2, n_igae_dec1, n_igae_dec2, n_clusters, device = device).to(device)






In [103]:
from utils import eva, target_distribution

acc_reuslt = []
nmi_result = []
ari_result = []
f1_result = []

def train(model, num_epoch, data, adj, label, lr, pre_model_save_path, final_model_save_path, 
          n_clusters, original_acc, gamma_value, lambda_value, device ):
    

    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    model.load_state_dict(torch.load(pre_model_save_path, map_location='cpu'))
    
    with torch.no_grad():
        xhat_ae, xhat_igae, adj_hat, z_ae, z_igae, _, _, _, z_tilde = model(data, adj)

    kmeans = KMeans(n_clusters=n_clusters, n_init=20)
    cluster_id = kmeans.fit_predict(z_tilde.data.cpu().numpy())
    model.cluster_layer.data = torch.tensor(kmeans.cluster_centers_).to(device)
    eva(label, cluster_id, 'Initialization')

    for epoch in range(num_epoch):

        x_hat, z_hat, adj_hat, z_ae, z_igae, q, q1, q2, z_tilde = model(data, adj)

        tmp_q = q.data
        p = target_distribution(tmp_q)

        loss_ae = F.mse_loss(x_hat, data)
        loss_w = F.mse_loss(z_hat, torch.spmm(adj, data))
        loss_a = F.mse_loss(adj_hat, adj.to_dense())
        loss_igae = loss_w + gamma_value * loss_a
        loss_kl = F.kl_div((q.log() + q1.log() + q2.log()) / 3, p, reduction='batchmean')
        loss = loss_ae + loss_igae + lambda_value * loss_kl
        print('{} loss: {}'.format(epoch, loss))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        kmeans = KMeans(n_clusters=n_clusters, n_init=20).fit(z_tilde.data.cpu().numpy())

        acc, nmi, ari, f1 = eva(label, kmeans.labels_, epoch)
        acc_reuslt.append(acc)
        nmi_result.append(nmi)
        ari_result.append(ari)
        f1_result.append(f1)

        if acc > original_acc:
            original_acc = acc
            torch.save(model.state_dict(), final_model_save_path)
    


In [104]:
torch.save(model.state_dict(), 'model_pretrain.pkl')

In [107]:
num_epoch = 100
lr = 5e-4
pre_model_save_path = 'model_pretrain.pkl'
final_model_save_path = 'model_final.pkl'



train(model, num_epoch,  X, adj, label, lr,  pre_model_save_path, final_model_save_path,
      n_clusters, -1, 0.1, 10, device)

Epoch_Initialization :acc 0.5078 , nmi 0.1642 , ari 0.1562 , f1 0.5005
0 loss: 0.2902512848377228
Epoch_0 :acc 0.5053 , nmi 0.1620 , ari 0.1535 , f1 0.4982
1 loss: 0.28229841589927673
Epoch_1 :acc 0.4996 , nmi 0.1537 , ari 0.1467 , f1 0.4904
2 loss: 0.2772531807422638
Epoch_2 :acc 0.4821 , nmi 0.1430 , ari 0.1304 , f1 0.4710
3 loss: 0.27427980303764343
Epoch_3 :acc 0.4673 , nmi 0.1377 , ari 0.1215 , f1 0.4569
4 loss: 0.27259185910224915
Epoch_4 :acc 0.4550 , nmi 0.1347 , ari 0.1151 , f1 0.4455
5 loss: 0.271635502576828
Epoch_5 :acc 0.4501 , nmi 0.1324 , ari 0.1116 , f1 0.4393
6 loss: 0.2710537314414978
Epoch_6 :acc 0.4493 , nmi 0.1332 , ari 0.1111 , f1 0.4386


KeyboardInterrupt: 